Download the dataset using cURL directly from kaggle.

In [8]:
%%bash
curl -L -o ~/Documents/GitHub/SQLcovid/corona-virus-report.zip \
  "https://www.kaggle.com/api/v1/datasets/download/imdevskp/corona-virus-report"


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 18.9M  100 18.9M    0     0  7056k      0  0:00:02  0:00:02 --:--:-- 10.1M


Unzip file into current folder.

In [9]:
%%bash
unzip -o ~/Documents/GitHub/SQLcovid/corona-virus-report.zip -d ~/Documents/GitHub/SQLcovid/

Archive:  /Users/joshuawb/Documents/GitHub/SQLcovid/corona-virus-report.zip
  inflating: /Users/joshuawb/Documents/GitHub/SQLcovid//country_wise_latest.csv  
  inflating: /Users/joshuawb/Documents/GitHub/SQLcovid//covid_19_clean_complete.csv  
  inflating: /Users/joshuawb/Documents/GitHub/SQLcovid//day_wise.csv  
  inflating: /Users/joshuawb/Documents/GitHub/SQLcovid//full_grouped.csv  
  inflating: /Users/joshuawb/Documents/GitHub/SQLcovid//usa_county_wise.csv  
  inflating: /Users/joshuawb/Documents/GitHub/SQLcovid//worldometer_data.csv  


We must pip install duckdb and pandas

In [13]:
import sys
!{sys.executable} -m pip install duckdb pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 51.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip3 install --upgrade pip


Using DuckDB, we are able to directly create a SQL table from our csv in one line.

In [41]:
import pandas as pd
import duckdb as ddb

con = ddb.connect() 
con.execute("CREATE TABLE COVID AS SELECT * FROM 'worldometer_data.csv'")

df = con.execute("SELECT * FROM COVID").df()

df.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
0,USA,North America,331198130,5032179,<NA>,162804,<NA>,2576668,<NA>,2292707,18296,15194,492.0,63139605,190640,Americas
1,Brazil,South America,212710692,2917562,<NA>,98644,<NA>,2047660,<NA>,771258,8318,13716,464.0,13206188,62085,Americas
2,India,Asia,1381344997,2025409,<NA>,41638,<NA>,1377384,<NA>,606387,8944,1466,30.0,22149351,16035,South-EastAsia
3,Russia,Europe,145940924,871894,<NA>,14606,<NA>,676357,<NA>,180931,2300,5974,100.0,29716907,203623,Europe
4,South Africa,Africa,59381566,538184,<NA>,9604,<NA>,387316,<NA>,141264,539,9063,162.0,3149807,53044,Africa


In [42]:
df.describe()

,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop
count,208.0,2.090000e+02,4.0,188.0,3.0,205.0,3.0,205.0,122.0,208.0,187.000000,191.0,191.0
mean,30415486.971154,9.171850e+04,1980.5,3792.590426,300.0,58878.980488,1706.0,27664.326829,534.393443,3196.024038,98.681176,1402404.701571,83959.366492
std,104766099.481173,4.325867e+05,3129.611424,15487.184877,451.199512,256698.406898,2154.779803,174632.736928,2047.518613,5191.986457,174.956862,5553366.670432,152730.59124
min,801.0,1.000000e+01,20.0,1.0,1.0,7.0,42.0,0.0,1.0,3.0,0.080000,61.0,4.0
25%,966314.0,7.120000e+02,27.5,22.0,40.5,334.0,489.0,86.0,3.25,282.0,6.000000,25752.0,8956.5
50%,7041972.5,4.491000e+03,656.0,113.0,80.0,2178.0,936.0,899.0,27.5,1015.0,29.000000,135702.0,32585.0
75%,25756135.5,3.689600e+04,2609.0,786.0,449.5,20553.0,2538.0,7124.0,160.25,3841.75,98.000000,757696.0,92154.5
max,1381344997.0,5.032179e+06,6590.0,162804.0,819.0,2576668.0,4140.0,2292707.0,18296.0,39922.0,1238.000000,63139605.0,995282.0


First 10 distinct countries in the dataset in alphabetical order.

In [49]:
con.execute("""
            SELECT C."Country/Region",  FROM COVID C
            ORDER BY 1
            LIMIT 10
            """).df()

,Country/Region
0,Afghanistan
1,Albania
2,Algeria
3,Andorra
4,Angola
5,Antigua and Barbuda
6,Argentina
7,Armenia
8,Aruba
9,Australia


Below we calculate the death rate across all countries and output the death rate for New Zealand. We can do this using a combination of pandas and SQL or do it full in SQL as shown in the next code block.

In [55]:
deathrates = con.execute("""
                            SELECT DISTINCT C."Country/Region", C.TotalCases, C.TotalDeaths, (TotalDeaths/TotalCases)*100 AS DeathRatePercentage
                            FROM COVID C
                        """).df()

deathrates.loc[lambda df: df["Country/Region"] == "New Zealand"]

,Country/Region,TotalCases,TotalDeaths,DeathRatePercentage
160,New Zealand,1569,22,1.402167


In [ ]:
con.execute("""
                SELECT DISTINCT C."Country/Region", C.TotalCases, C.TotalDeaths, (TotalDeaths/TotalCases)*100 AS DeathRatePercentage
                FROM COVID C
                WHERE C."Country/Region" = 'New Zealand'
            """).df()

,Country/Region,TotalCases,TotalDeaths,DeathRatePercentage
0,New Zealand,1569,22,1.402167


We also might like to see which countries had the highest death rate.

In [57]:
con.execute("""
                SELECT DISTINCT C."Country/Region", C.TotalCases, C.TotalDeaths, ROUND((TotalDeaths/TotalCases)*100,2) AS DeathRatePercentage
                FROM COVID C
                ORDER BY DeathRatePercentage DESC
                LIMIT 10
            """).df()

,Country/Region,TotalCases,TotalDeaths,DeathRatePercentage
0,Yemen,1768,508,28.73
1,France,195633,30312,15.49
2,UK,308134,46413,15.06
3,Italy,249204,35187,14.12
4,Belgium,71158,9859,13.86
5,Hungary,4597,600,13.05
6,Mexico,462690,50517,10.92
7,Netherlands,56982,6153,10.80
8,Sint Maarten,160,16,10.00
9,Western Sahara,10,1,10.00


In [ ]:
con.execute("""
            SELECT C."Country/Region",  FROM COVID C
            ORDER BY 1
            LIMIT 10
            """).df()